In [1]:
import numpy as np
import pandas as pd
import random
import warnings
warnings.filterwarnings("ignore")

In [2]:
data = pd.read_csv("Datafiniti_Fast_Food_Restaurants_Jun19.csv")[:2000]

In [3]:
data['postalCode'] = data['postalCode'].apply(lambda x: x[:5])

data = data.set_index(['address','postalCode'])
remove_indices = [i for i in data.index if data.index.value_counts()[i]>1]
data = data.drop(remove_indices)
data = data.reset_index()

In [4]:
location = data[['address','name','postalCode']]
outlet = data[['dateAdded']]
city = data[['postalCode','city','province']]

In [5]:
city = city.set_index('postalCode')
city = city.drop_duplicates()
city = city.reset_index()
city = city.rename(columns={'postalCode':'zipcode', 'province':'state'})

city.head()

,zipcode,city,state
0,80631,Greeley,CO
1,81505,Grand Junction,CO
2,80011,Aurora,CO
3,80220,Denver,CO
4,72703,Fayetteville,AR


In [6]:
location['contact'] = 0
for i in range(len(location)):
    location.loc[i,'contact'] = random.randint(1010000000,9990000000)
location = location.rename(columns={'address':'street_address','postalCode':'zipcode'})
location['name'] = location['name'].apply(lambda x: x.upper())
location['name'] = location['name'].apply(lambda x: x.replace("'",""))

In [7]:
franchise = pd.read_csv('Franchise.csv')
franchise.head()

,fran_id,name,category,website
0,1,MCDONALDS,BURGER,https://www.mcdonalds.com/us/en-us.html
1,2,STARBUCKS,SNACK,https://www.starbucks.com
2,3,SUBWAY,SANDWICH,https://order.subway.com/en-US
3,4,TACO BELL,GLOBAL,https://www.tacobell.com
4,5,CHICK-FIL-A,CHICKEN,https://www.chick-fil-a.com


In [9]:
loc_names = set(location.name.unique())
fran_names = set(franchise.name.unique())
remaining = list(fran_names - loc_names)
location['name'] = location['name'].apply(lambda x: x if x in fran_names else random.choice(remaining))

franchise = franchise.set_index('name')
location['fran_id'] = np.nan
for i in range(len(location)):
    location.loc[i,'fran_id'] = franchise.loc[location.loc[i,'name'], 'fran_id']
franchise = franchise.reset_index()

location['outlet_id'] = list(range(1,len(location)+1))
    
location = location[['outlet_id','street_address','zipcode','contact','fran_id']]
location = location.astype({'zipcode':'int64', 'fran_id':'int64'})

In [10]:
outlet = outlet.rename(columns={'dateAdded':'open_date'})
outlet['open_date'] = outlet['open_date'].apply(lambda x: x[:10])

outlet['employee_count'] = 0
outlet['parking'] = False
for i in range(len(outlet)):
    outlet.loc[i,'employee_count'] = random.randint(50,120)
    outlet.loc[i,'parking'] = bool(random.getrandbits(1))
    
outlet['outlet_id'] = list(range(1,len(outlet)+1))

outlet['manager_id'] = list(range(1,len(outlet)+1))
random.shuffle(outlet['manager_id'])

outlet = outlet[['outlet_id','open_date','employee_count','parking','manager_id']]

In [11]:
outlet

,outlet_id,open_date,employee_count,parking,manager_id
0,1,2019-05-19,106,True,1312
1,2,2018-01-18,87,True,1524
2,3,2019-05-19,82,True,1421
3,4,2019-05-19,81,True,1549
4,5,2019-05-19,88,True,1195
...,...,...,...,...,...
1981,1982,2016-11-14,74,True,402
1982,1983,2016-03-24,78,False,558
1983,1984,2015-11-29,113,False,952
1984,1985,2017-07-19,51,True,1158


In [12]:
location

,outlet_id,street_address,zipcode,contact,fran_id
0,1,2555 11th Avenue,80631,3704364173,28
1,2,2513 Highway 6 And 50,81505,2575716456,38
2,3,1125 Patterson Road,81506,4415599384,12
3,4,3455 N Salida Court,80011,4687789204,37
4,5,5225 E Colfax Avenue,80220,4682777813,4
...,...,...,...,...,...
1981,1982,9800 76th St,53158,1655882637,19
1982,1983,615 W Lake St,60661,1292286488,3
1983,1984,8101 Broadway,46410,7512538727,21
1984,1985,369 S Avalon Park Blvd,32828,9576390682,4


In [13]:
city

,zipcode,city,state
0,80631,Greeley,CO
1,81505,Grand Junction,CO
2,80011,Aurora,CO
3,80220,Denver,CO
4,72703,Fayetteville,AR
...,...,...,...
1039,63383,Warrenton,MO
1040,62052,Jerseyville,IL
1041,61270,Morrison,IL
1042,53158,Pleasant Prairie,WI


In [18]:
outlet.to_csv('Outlet.csv', index=False)
location.to_csv('Location.csv', index=False)
city.to_csv('City.csv', index=False)